In [1]:
%config IPCompleter.greedy=True  #Tab代码自动提示

### 引入依赖

In [2]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

In [3]:
tf.version.VERSION

'2.1.0'

## 调用GPU

In [4]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0
tf.config.experimental.set_memory_growth(physical_devices[0], True)

## 载入数据
用wget下载，这个url需要科学上网

In [4]:
import wget

# 测试集
url = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip'
target = './tmp/validation-horse-or-human.zip'

file_name = wget.download(url, out = target)

# 训练集
url = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip'
target = './tmp/horse-or-human.zip'

file_name = wget.download(url, out = target)

# 第四部分 更复杂的图像识别
## 4.2 ImageDataGenerator

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
# 创建两个数据生成器，指定scaling范围0-1
train_datagen = ImageDataGenerator(rescale = 1 / 255)
validation_datagen = ImageDataGenerator(rescale = 1 / 255)

In [7]:
# 指向训练数据文件夹
train_generator = train_datagen.flow_from_directory(
    './tmp/horse-or-human/',    # 训练数据所在的文件夹
    target_size = (300, 300),   # 指定输出尺寸
    batch_size = 32,
    class_mode = 'binary'       # 指定二分类
)

# 指向测试数据文件夹
validation_generator = validation_datagen.flow_from_directory(
    './tmp/validation-horse-or-human/',
    target_size = (300, 300),
    batch_size = 32,
    class_mode = 'binary'
)


Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


## 4.3 构建并训练模型

In [8]:
model = keras.models.Sequential([
    keras.layers.Conv2D(16, (3, 3), activation = 'relu', 
                        input_shape = (300, 300, 3)),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(64, (3, 3), activation = 'relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 78400)             0

In [9]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
              metrics=['acc']
)

In [10]:
history = model.fit(
    train_generator,
    epochs=15,
    verbose=1,
    validation_data=validation_generator,
    validation_steps=8
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 33 steps, validate for 8 steps
Epoch 1/15
33/33 [==============================] - 7s 207ms/step - loss: 1.0132 - acc: 0.7936 - val_loss: 2.4316 - val_acc: 0.6094
Epoch 2/15
33/33 [==============================] - 4s 131ms/step - loss: 0.1492 - acc: 0.9445 - val_loss: 5.1020 - val_acc: 0.6055
Epoch 3/15
33/33 [==============================] - 4s 131ms/step - loss: 0.0986 - acc: 0.9688 - val_loss: 4.1372 - val_acc: 0.6367
Epoch 4/15
33/33 [==============================] - 4s 131ms/step - loss: 0.1244 - acc: 0.9542 - val_loss: 2.0506 - val_acc: 0.8047
Epoch 5/15
33/33 [==============================] - 4s 132ms/step - loss: 0.0183 - acc: 0.9942 - val_loss: 2.4656 - val_acc: 0.8086
Epoch 6/15
33/33 [==============================] - 4s 131ms/step - loss: 0.6976 - acc: 0.9903 - val_loss: 4.5152 - val_acc: 0.7422
Epoch 7/15
33/33 [==============================] - 4s 131ms/step - loss: 1.5327e-04 - acc: 1.0000 - val_loss: 4.9781

## 4.4 优化模型参数

In [8]:
from kerastuner.tuners import Hyperband
from kerastuner.engine.hyperparameters import HyperParameters

In [9]:
hp = HyperParameters()

In [13]:
def build_model(hp):
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(
        hp.Choice('input_layer0',values=[16,64],default=16),
        (3, 3), 
        activation = 'relu', 
        input_shape = (300, 300, 3)
    ))
    model.add(keras.layers.MaxPooling2D(2, 2))
        
    for i in range(hp.Int('num_conv_layers',1,3)):
        model.add(keras.layers.Conv2D(
            hp.Choice('conv2d_layer{0}'.format(i), values=[16,64],default=16), 
            (3, 3), 
            activation = 'relu'
        ))
        model.add(keras.layers.MaxPooling2D(2, 2))
    
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(hp.Int('dense_layer0',128,512,step=32), activation = 'relu'))
    model.add(keras.layers.Dense(1, activation = 'sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
                  metrics=['acc']
    )
    model.summary()
    return model


In [14]:
tuner = Hyperband(
    build_model,
    objective='val_acc',
    max_epochs=15,
    directory='params',
    hyperparameters=hp,
    project_name='horse_or_human'
)

tuner.search(
    train_generator,
    epochs=15,
    verbose=1,
    validation_data=validation_generator,
    validation_steps=8
)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 16)      2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 85264)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               10913920  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 341056)            0         
_________________________________________________________________
dense (Dense)                (None, 416)               141879712 
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 4

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 16)      9232      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 16)        2320      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 16)        2

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 16)      2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 16)        2320      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 19600)             0

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 78400)             0

...

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 64)      9280      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 78400)             0

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 64)      9280      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 16)        9232      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 19600)             0

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 78400)             0

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 16)      9232      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        9280      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 16)        9

INFO:tensorflow:Oracle triggered exit


In [15]:
best_hps = tuner.get_best_hyperparameters(1)[0]
print(best_hps.values)

{'input_layer0': 64, 'num_conv_layers': 3, 'conv2d_layer0': 16, 'dense_layer0': 256, 'conv2d_layer1': 64, 'conv2d_layer2': 16, 'tuner/epochs': 15, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}


In [18]:
model = tuner.hypermodel.build(best_hps)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 16)      9232      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        9280      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 16)        9